In [1]:
import numpy as np
import pandas as pd
import io
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Load the data
from google.colab import files
uploaded = files.upload()

Saving rawdata.csv to rawdata.csv


In [4]:
# Get the data into dataframe
data = io.BytesIO(uploaded['rawdata.csv'])   
df_raw = pd.read_csv(data)
print(df_raw.shape)
df_raw.head(10)

(10919, 6)


,TransitionNum,Q1,Q3,MT,Area,Sample
0,0,115.3,87.1,10.88,11070312.0,01_G1_1_U
1,1,60.9,44.0,11.13,1100433.0,01_G1_1_U
2,2,74.1,57.0,5.55,17784.0,01_G1_1_U
3,3,76.1,59.2,10.63,266665.0,01_G1_1_U
4,4,86.1,41.1,NaN,NaN,01_G1_1_U
5,5,89.1,72.0,10.97,43021.0,01_G1_1_U
6,6,90.0,44.0,NaN,NaN,01_G1_1_U
7,7,100.0,55.1,NaN,NaN,01_G1_1_U
8,8,101.0,83.1,NaN,NaN,01_G1_1_U
9,9,104.1,60.1,11.04,9370.0,01_G1_1_U


In [5]:
# Fill in the NaN to 0 temporarily
df = df_raw.fillna(0)
df.head(10)

,TransitionNum,Q1,Q3,MT,Area,Sample
0,0,115.3,87.1,10.88,11070312.0,01_G1_1_U
1,1,60.9,44.0,11.13,1100433.0,01_G1_1_U
2,2,74.1,57.0,5.55,17784.0,01_G1_1_U
3,3,76.1,59.2,10.63,266665.0,01_G1_1_U
4,4,86.1,41.1,0.00,0.0,01_G1_1_U
5,5,89.1,72.0,10.97,43021.0,01_G1_1_U
6,6,90.0,44.0,0.00,0.0,01_G1_1_U
7,7,100.0,55.1,0.00,0.0,01_G1_1_U
8,8,101.0,83.1,0.00,0.0,01_G1_1_U
9,9,104.1,60.1,11.04,9370.0,01_G1_1_U


In [6]:
# Check the sample number and the peak number for each sample
df_no_zero = df[df['Area']>0].reset_index(drop=True)
sample_name = df_no_zero['Sample'].unique()
print('There are {} samples.'.format(len(sample_name)))
print(' ')
print('the peak numbers for each sample are as follow:')
print (df_no_zero['Sample'].value_counts())

There are 32 samples.
 
the peak numbers for each sample are as follow:
14_G5_2_U    382
29_G2_3_R    358
23_G3_3_S    341
22_G3_2_S    340
32_G4_3_R    337
28_G2_2_R    335
31_G4_2_R    330
27_G2_1_R    320
08_G3_2_U    318
17_G1_3_S    309
01_G1_1_U    294
16_G1_2_S    284
21_G3_1_S    282
09_G3_3_U    275
30_G4_1_R    272
06_G2_3_U    271
15_G1_1_S    267
02_G1_2_U    263
05_G2_2_U    260
07_G3_1_U    259
10_G4_1_U    251
24_G4_1_S    250
18_G2_1_S    246
13_G5_1_U    245
04_G2_1_U    244
19_G2_2_S    235
25_G4_2_S    226
12_G4_3_U    220
03_G1_3_U    219
11_G4_2_U    209
26_G4_3_S    193
20_G2_3_S    174
Name: Sample, dtype: int64


In [7]:
# Change the sample name for simplicity
df_no_zero['sample'] = df_no_zero['Sample'].apply(lambda x:x[:2]).tolist()
df_no_zero['type'] = df_no_zero['Sample'].apply(lambda x:x[-1]).tolist()
df2 = df_no_zero.drop(['Sample'], axis=1)
df2.head(10)

,TransitionNum,Q1,Q3,MT,Area,sample,type
0,0,115.3,87.1,10.88,11070312.0,01,U
1,1,60.9,44.0,11.13,1100433.0,01,U
2,2,74.1,57.0,5.55,17784.0,01,U
3,3,76.1,59.2,10.63,266665.0,01,U
4,5,89.1,72.0,10.97,43021.0,01,U
5,9,104.1,60.1,11.04,9370.0,01,U
6,10,104.1,87.0,11.04,14992.0,01,U
7,11,112.1,94.9,10.91,128602.0,01,U
8,12,114.3,86.1,10.91,14201720.0,01,U
9,13,115.0,73.0,10.94,489144.0,01,U


In [8]:
# Define functions for dividing the dataset
def divide_data(sample_number, source_df):
    dfs = {}
    types = {}
    i=0
    j=1
    for i in range(sample_number):
        i_str = str(i+1)       
        i_str_2digits = i_str.zfill(2)
        dfs[j] = source_df.groupby(['sample']).get_group(i_str_2digits).reset_index(drop=True)
        ty = dfs[j].loc[0,'type']
        dfs[j] = dfs[j].drop(columns=['Q1','Q3','sample','type'])
        dfs[j] = dfs[j].rename(columns={'TransitionNum':'num',
                                        'MT': i_str+'_time', 
                                        'Area':i_str+'_area'}).reset_index(drop=True)
        types[j] = str(ty)
        i += 1
        j += 1
    return dfs, types

In [9]:
# Create datasets for each sample
num = len(sample_name)
df_samples, types = divide_data(num,df2)
print(types)
df_samples[1]

{1: 'U', 2: 'U', 3: 'U', 4: 'U', 5: 'U', 6: 'U', 7: 'U', 8: 'U', 9: 'U', 10: 'U', 11: 'U', 12: 'U', 13: 'U', 14: 'U', 15: 'S', 16: 'S', 17: 'S', 18: 'S', 19: 'S', 20: 'S', 21: 'S', 22: 'S', 23: 'S', 24: 'S', 25: 'S', 26: 'S', 27: 'R', 28: 'R', 29: 'R', 30: 'R', 31: 'R', 32: 'R'}


,num,1_time,1_area
0,0,10.88,11070312.0
1,1,11.13,1100433.0
2,2,5.55,17784.0
3,3,10.63,266665.0
4,5,10.97,43021.0
...,...,...,...
289,258,18.92,16320000.0
290,259,6.30,8428.0
291,259,23.29,9360.0
292,260,19.05,104005.0


In [32]:
# Define a function for suggesting the potential migration time markers for phase 1 and phase 2
def find_markers(df_samples):
    num_of_transitions = 0
    for i in range(1,len(df_samples)+1):
        if df_samples[i]['num'].max()>num_of_transitions:
            num_of_transitions=df_samples[i]['num'].max()
    one_peak=pd.DataFrame({'contains_one_peak':[True]*(num_of_transitions+1)},
                          index=range(0,(num_of_transitions+1)))
    for i in range(1,len(df_samples)+1):
        num_is_one=df_samples[i].groupby(['num'])['num'].count()==1
        for j in range(0,num_of_transitions+1):
            if j not in num_is_one.index or not num_is_one[j]:
                one_peak['contains_one_peak'][j]=False
    selected=one_peak.loc[one_peak['contains_one_peak']].index
    res=df_samples[1].set_index('num').loc[selected]
    res=res.drop('1_area',axis='columns')
    for i in range(2,len(df_samples)+1):
        right=df_samples[i].set_index('num').loc[selected]
        right=right.drop(str(i)+'_area',axis='columns')
        res=res.join(right)
    res['Average']=res.mean(axis=1)
    res['std']=res.std(axis=1)
    res['RSD(%)']=res['std']/res['Average']*100
    res = res.drop(['std'], axis=1)
    res=res.sort_values(by='Average').round(2)
    return res

In [34]:
# Find the first four markers based on two conditions that (1) only one peak is found
# in that transition(Q1/Q3) and (2) this analyte can be found in all samples.
potential_markers = find_markers(df_samples)
potential_markers.head(31)

,1_time,2_time,3_time,4_time,5_time,6_time,7_time,8_time,9_time,10_time,11_time,12_time,13_time,14_time,15_time,16_time,17_time,18_time,19_time,20_time,21_time,22_time,23_time,24_time,25_time,26_time,27_time,28_time,29_time,30_time,31_time,32_time,Average,RSD(%)
num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
253,4.83,5.30,4.58,4.86,5.08,5.14,5.27,4.86,5.05,5.24,5.14,5.14,5.21,4.89,4.86,4.43,5.01,5.21,5.39,5.18,5.08,4.89,5.36,4.49,4.68,5.11,5.39,4.86,5.24,5.25,5.14,5.31,5.05,4.98
37,5.49,5.92,5.02,5.45,5.61,5.77,5.92,5.55,5.58,5.99,5.77,5.61,5.77,5.58,5.49,5.08,5.70,6.02,6.27,5.71,5.83,5.55,6.05,4.99,5.30,5.76,5.95,5.61,5.92,5.83,5.80,5.89,5.68,5.10
38,5.61,6.27,5.30,5.67,5.86,6.20,6.30,5.77,5.89,6.27,6.08,5.95,6.05,5.86,5.86,5.24,5.92,6.33,6.55,5.95,6.08,5.80,6.27,5.21,5.58,6.02,6.23,5.80,6.20,6.20,6.05,6.20,5.96,5.33
27,6.67,7.57,6.36,6.86,7.23,7.42,7.64,6.83,7.14,7.51,7.36,7.29,7.33,6.83,6.95,6.30,7.14,7.64,7.92,7.26,7.39,6.86,7.54,6.27,6.76,7.26,7.57,6.92,7.48,7.39,7.26,7.51,7.17,5.64
125,10.32,9.57,7.23,7.92,8.45,8.57,8.82,9.42,8.17,9.60,8.70,8.48,8.88,9.60,8.23,7.14,8.14,9.63,9.94,8.64,8.54,9.54,9.32,7.20,8.20,8.79,8.76,8.85,8.57,8.60,8.48,8.54,8.71,8.57
60,10.63,13.34,10.13,11.16,12.10,12.10,12.94,11.10,11.44,13.28,12.56,12.06,12.47,11.19,11.78,9.98,11.38,13.34,13.75,12.22,12.47,11.16,13.28,10.04,11.28,12.22,12.84,11.19,12.09,12.19,12.25,12.19,11.94,8.18
3,10.63,13.34,10.13,11.22,12.06,12.13,12.94,11.10,11.44,13.28,12.56,12.06,12.50,11.22,11.78,9.98,11.38,13.37,13.72,12.22,12.50,11.13,13.28,10.04,11.28,12.22,12.84,11.22,12.09,12.22,12.25,12.22,11.95,8.17
36,10.69,13.50,10.32,11.25,12.19,12.19,13.06,11.16,11.57,13.37,12.62,12.19,12.56,11.29,11.91,10.10,11.50,13.53,13.87,12.28,12.59,11.25,13.37,10.10,11.35,12.28,12.94,11.25,12.16,12.41,12.34,12.28,12.05,8.17
39,10.72,13.72,10.41,11.47,12.38,12.50,13.59,11.38,11.75,13.65,12.94,12.50,12.78,11.47,12.06,10.29,11.75,13.72,14.03,12.44,13.12,11.47,13.65,10.22,11.50,12.47,13.40,11.44,12.56,12.66,12.66,12.66,12.29,8.49
